This notebook tests various scenarios that are specific to notebook scope (and therefore are not covered under the regular unit tests).

In [ ]:
%pip install -qU pixeltable

In [ ]:
import pixeltable as pxt

pxt.drop_dir('tests', force=True)
pxt.create_dir('tests')
t = pxt.create_table('tests.notebook_test_tbl', {'int_col': pxt.Int})
t.insert(int_col=1)

In [ ]:
# Test that UDFs can be redefined in notebook scope, and that computed columns use the definition that
# was in force at the time the column was created.

@pxt.udf
def udf1(x: int) -> int:
    return x + 1

t.add_column(int_col_plus_1=udf1(t.int_col))

@pxt.udf
def udf1(x: int) -> int:
    return x + 2

t.add_column(int_col_plus_2=udf1(t.int_col))

row = t.select(t.int_col, t.int_col_plus_1, t.int_col_plus_2).collect()[0]
assert list(row.values()) == [1, 2, 3]

In [3]:
# Test that stored queries can be redefined in notebook scope.

@pxt.query
def query1(x: int):
    return t.select(t.int_col == x)

@pxt.query
def query1(x: int):
    return t.select(t.int_col == x + 1)